<div align="right" style="text-align:right"><a rel="license" href="http://creativecommons.org/licenses/by-nc-nd/4.0/"><img alt="Licença Creative Commons" style="border-width:0; float:right" src="https://i.creativecommons.org/l/by-nc-nd/4.0/88x31.png" /></a><br><br><i>Prof. Marcelo de Souza</i><br>marcelo.desouza@udesc.br</div>

# Problema do transporte (Hitchcock)

Programação linear com Pyomo.

---

Uma empresa tem $m$ depósitos, cada um com estoque $a_i$, $i \in [m]$ toneladas de determinado produto. Há pedidos de $n$ clientes de $b_j$, $j \in [n]$ toneladas do produto. O transporte de uma tonelada do produto de um depósito $i$ para um cliente $j$ custa $c_{ij}$.

O problema do transporte consiste em determinar o esquema de transporte de menor custo para atender aos pedidos.

---

Consideremos uma instância concreta do problema, representada pelo grafo abaixo (esquerda). Os valores dos vértices definem o estoque dos depósitos e a demanda dos clientes. Os pesos nas arestas indicam o custo de transporte dos depósitos aos clientes.

<img src="./transporte.png" width="500px"/>

O lado direito da figura apresenta uma solução para a instância, onde os arcos representam o transporte do produto e seus pesos indicam a quantidade transportada.

***

Preparação para execução no Google Colab

In [ ]:
!pip install -qq pyomo
!apt-get install -y -qq glpk-utils

Podemos construir um modelo para resolver a instância acima. Para isso definimos:
+ Variáveis de decisão $x_{ij}$ que determinam a quantidade transportada do depósito $i$ ao cliente $j$;
+ Restrições de estoque, que limitam a quantidade que pode ser transportada de cada depósito;
+ Restrições de demanda, que estabelecem a quantidade que cada cliente deve receber;
+ Adicionalmente, restringimos o transporte de pares {depósito, cliente} onde não há previsão de transporte.

In [1]:
from pyomo.environ import *

# Criação do modelo
model = ConcreteModel()

# Variáveis de decisão
model.x11 = Var(domain = NonNegativeReals)
model.x12 = Var(domain = NonNegativeReals)
model.x13 = Var(domain = NonNegativeReals)
model.x21 = Var(domain = NonNegativeReals)
model.x22 = Var(domain = NonNegativeReals)
model.x23 = Var(domain = NonNegativeReals)
model.x31 = Var(domain = NonNegativeReals)
model.x32 = Var(domain = NonNegativeReals)
model.x33 = Var(domain = NonNegativeReals)

# Função objetivo
#model.obj = Objective(expr = 3 * model.x11 + 1 * model.x12 + 100 * model.x13 + 4 * model.x21 + 2 * model.x22 + 4 * model.x23 + 100 * model.x31 + 3 * model.x32 + 3 * model.x33, sense = minimize)
model.obj = Objective(expr = 3 * model.x11 + 1 * model.x12 + 4 * model.x21 + 2 * model.x22 + 4 * model.x23 + 3 * model.x32 + 3 * model.x33, sense = minimize)

# Restrições de estoque
model.con1 = Constraint(expr = model.x11 + model.x12 + model.x13 <= 5)
model.con2 = Constraint(expr = model.x21 + model.x22 + model.x23 <= 7)
model.con3 = Constraint(expr = model.x31 + model.x32 + model.x33 <= 3)

# Restrições de demanda
model.con4 = Constraint(expr = model.x11 + model.x21 + model.x31 == 7)
model.con5 = Constraint(expr = model.x12 + model.x22 + model.x32 == 3)
model.con6 = Constraint(expr = model.x13 + model.x23 + model.x33 == 5)

# Restrições de pares depósito x cliente
model.con7 = Constraint(expr = model.x13 == 0)
model.con8 = Constraint(expr = model.x31 == 0)

# Solução
opt = SolverFactory('glpk', executable='/usr/bin/glpsol')
opt.solve(model).write()
print()
print('Custo total:', model.obj())
print('1 --> 1:', model.x11())
print('1 --> 2:', model.x12())
print('1 --> 3:', model.x13())
print('2 --> 1:', model.x21())
print('2 --> 2:', model.x22())
print('2 --> 3:', model.x23())
print('3 --> 1:', model.x31())
print('3 --> 2:', model.x32())
print('3 --> 3:', model.x33())

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 46.0
  Upper bound: 46.0
  Number of objectives: 1
  Number of constraints: 9
  Number of variables: 10
  Number of nonzeros: 21
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.0073544979095458984
# ----------------------------------------------------------
#   Solution Information
# -----------------------------------

***

Também podemos criar um modelo genérico para resolver qualquer instância do problema do transporte:

$$
\begin{aligned}
    \text{minimiza} \quad z = & \sum_{i \in [m], j \in [n]} c_{ij} x_{ij}\\[.3em]
    \text{sujeito a} \quad & \sum_{j \in [n]} x_{ij} \le a_i, \quad\text{para todo fornecedor }i \in [m]\\
              & \sum_{i \in [m]} x_{ij} = b_j, \quad\text{para todo cliente }j \in [n]\\
              & x_{ij} \ge 0, \quad\text{para todo fornecedor }i \in [m]\text{ e todo cliente }j \in [n]\\
\end{aligned}
$$

Dados:

In [2]:
m = 3                # fornecedores
n = 3                # clientes
a = [5, 7, 3]        # estoques
b = [7, 3, 5]        # demandas
c = [[3, 1, 100],    # custos
     [4, 2, 4],
     [100, 3, 3]]

Implementação do modelo:

In [3]:
from pyomo.environ import *

# Criação do modelo
model = ConcreteModel()

# Variáveis de decisão: para cada par depósito x cliente
model.x = Var([i for i in range(m)], [j for j in range(n)], domain = NonNegativeReals)

def objective_function(model):
    result = 0
    for i in range(m):
        for j in range(n):
            result += model.x[i,j] * c[i][j]
    return result

# Função objetivo
#model.obj = Objective(expr = sum(model.x[i,j]*c[i][j] for i in range(m) for j in range(n)))
model.obj = Objective(rule = objective_function)

# Restrições
model.cons = ConstraintList()

for i in range(m):
    model.cons.add(expr = sum(model.x[i,j] for j in range(n)) <= a[i])

for j in range(n):
    model.cons.add(expr = sum(model.x[i,j] for i in range(m)) == b[j])

# Solução
solver = SolverFactory('glpk')
solver.solve(model).write()

print('Custo total:', model.obj())
for i in range(m):
    for j in range(n):
        print(i + 1, '-->', j + 1, ':', model.x[i,j]())

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 46.0
  Upper bound: 46.0
  Number of objectives: 1
  Number of constraints: 7
  Number of variables: 10
  Number of nonzeros: 19
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.028554677963256836
# ----------------------------------------------------------
#   Solution Information
# ------------------------------------